In [ ]:
%%capture
# pip install gdown
# !gdown --id 1qrDgGHspmsoqS5ZvGNzr-KVE0MVrHdlP
!gdown --id 1mRzWOHbXy3od2Qrv3e2K3CF2a36Q3lxW
# !unzip -q Images_data.zip -d all_data
# !rm -r Images_data.zip

In [ ]:
!unzip -q '/content/drive/MyDrive/Colab Notebooks/Fashion Search Engine Project/Data/Images_data.zip' -d all_data

In [ ]:
import os
import json
import shutil
import zipfile
from PIL import Image
from tqdm import tqdm

project_path = '/content/project'
labels_path   = '/content' + '/Lables.json'
src_img_path  = project_path + '/original_images'
dist_img_path = project_path + '/extracted_images'
os.makedirs(dist_img_path, exist_ok=True)

shutil.copytree("/content/all_data/NS_Data",src_img_path)

print(f"Number of images in the folder: {len(os.listdir(src_img_path))}")
print(f"Number of images in the dist_img_path: {len(os.listdir(dist_img_path))}")

Number of images in the folder: 46089
Number of images in the dist_img_path: 0


In [ ]:
def rename_files(folder_path):
    file_map = {}

    for i, filename in enumerate(os.listdir(folder_path)):

        # Generate the new name by combining the new prefix and the file's extension
        file_extension = os.path.splitext(filename)[1]
        new_name = f"{filename[:5]}_{i}{file_extension}"

        # Rename the file
        old_path = os.path.join(folder_path, filename)
        new_path = os.path.join(folder_path, new_name)
        os.rename(old_path, new_path)

        # Store the mapping of old name to new name
        file_map[filename] = new_name

    return file_map
# Call the function to rename the files
file_map = rename_files(src_img_path)

In [ ]:
# task >> eny 23ed run l bta3a deh
fashion_categories = [
    'Suitcase', 'Miniskirt', 'Tie', 'Luggage & bags', 'Shoe', 'Belt', 'Outerwear', 'Dress',
    'Earrings', 'Bracelet', 'Necklace', 'Brassiere', 'Footwear',
    'Satchel', 'Bowtie', 'Top', 'Pants', 'Sunglasses', 'Swimwear', 'Clothing',
    'Glove', 'Skirt', 'High heels', 'Underpants',
    'Fedora','Sun hat', 'Sock',
    'Wallet', 'Scarf', 'Watch', 'Umbrella', 'Glasses',
    'Boot', 'Basket', 'Backpack', 'Bag',
    'Hat', 'Coat', 'Sandal', 'Shorts', 'Jeans', 'Shirt', 'Handbag', 'Jacket'
]

def extract_items_from_annotations_image_based(json_file, image_directory, output_directory):
    with open(json_file, 'r') as f:
        annotations = json.load(f)
        cnt=0

    for image_filename, items in tqdm(annotations.items()):
        image_filename = file_map[image_filename]
        cnt+=1

        image_path = os.path.join(image_directory, image_filename)
        image = Image.open(image_path)

        label_counts = {}  # To keep track of label occurrences within the image
        for item in items:
            for label, coordinates in item.items():
                if label not in fashion_categories:
                    continue  # Skip unrelated labels

                if label not in label_counts:
                    label_counts[label] = 1
                    label_id = label
                else:
                    label_counts[label] += 1
                    label_id = f"{label}_{label_counts[label]}"

                x_min = int(coordinates[0] * image.width)
                y_min = int(coordinates[1] * image.height)
                x_max = int(coordinates[2] * image.width)
                y_max = int(coordinates[3] * image.height)

                item_image = image.crop((x_min, y_min, x_max, y_max))

                # Create subfolder for the image if it doesn't exist
                image_output_directory = os.path.join(output_directory, image_filename.split('.')[0])
                os.makedirs(image_output_directory, exist_ok=True)

                output_filename = f"{label_id}_{image_filename}"
                output_path = os.path.join(image_output_directory, output_filename)
                item_image.save(output_path)
    print(f'image skipped(Non Fashiom Image) : {cnt}')


In [ ]:
extract_items_from_annotations_image_based(labels_path, src_img_path, dist_img_path)

100%|██████████| 46089/46089 [22:55<00:00, 33.51it/s]


image skipped(Non Fashiom Image) : 46089


In [ ]:
print(f"Number of images in the dist_img_path: {len(os.listdir(dist_img_path))}")

Number of images in the dist_img_path: 44860


In [ ]:
# Zip file path
zip_file_path = "/content/fashion_dataset/fashion_data"

# Compress the src_img_path folder
shutil.make_archive(zip_file_path, 'zip', project_path)

# # Remove the temporary directory
# shutil.rmtree(project_path)
# shutil.rmtree(src_img_path)
# shutil.rmtree(dist_img_path)
# shutil.rmtree('all_data')
# os.remove(labels_path)


print("Folders 'original_images' and 'extracted_images' compressed successfully as 'fashion_data.zip'.")

Folders 'original_images' and 'extracted_images' compressed successfully as 'fashion_data.zip'.


In [ ]:
# !cp '/content/fashion_data.zip' '/content/drive/MyDrive/Colab Notebooks/Fashion Search Engine Project/Data/fashion_dataset.zip'

In [ ]:
# !pip install kaggle
# !mkdir ~/.kaggle
# !mv ./kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!kaggle datasets init -p /content/fashion_dataset

Data package template written to: /content/fashion_dataset/dataset-metadata.json


In [ ]:
!kaggle datasets create -p /content/fashion_dataset

Starting upload for file fashion_data.zip
100% 5.36G/5.36G [00:58<00:00, 98.9MB/s]
Upload successful: fashion_data.zip (5GB)
Your private Dataset is being created. Please check progress at https://www.kaggle.com/datasets/mostafafathy4869/fashion-dataset22


In [ ]:
import kaggle

# Specify the source file(s) or directory that you want to include in the dataset
source_file = '/content/fashion_data.zip'
# You can include multiple files or directories by using a list:
# source_files = ['path/to/your/source/file1.csv', 'path/to/your/source/file2.csv']

# Specify the destination folder and the dataset name
destination_folder = 'path/to/your/dataset'
dataset_name = 'your-dataset-name'

# Create the dataset using the kaggle datasets create command
kaggle.api.dataset_create_new(destination_folder, dataset_name)

# Add the files to the dataset using the kaggle datasets version command
kaggle.api.dataset_create_version(destination_folder, dataset_name, version_notes='Initial version', file_paths=source_file)
